In [1]:
!pip install pdfminer.six

In [2]:
import io
import re
from pdfminer.high_level import extract_text_to_fp, extract_pages
from pdfminer.layout import LAParams

In [51]:
def extract_info(file_path):
    # Create a buffer to store the extracted text
    buf = io.StringIO()
    
    # Extract the text using pdfminer
    with open(file_path, 'rb') as fp:
        extract_text_to_fp(fp, buf, laparams=LAParams())
    
    # Preprocess the text to remove unnecessary whitespace and line breaks
    text = buf.getvalue().strip()

    # Exclude pages that contain "DAFTAR ISI" or "DAFTAR TABEL"
    pages = text.split('\x0c')
    exclude_list = ["DAFTAR ISI", "DAFTAR TABEL", "PRAKATA", "PENDAHULUAN", "LAMPIRAN I", 
                    "Format 1", "Format 4", "Format 5", "Format 6", "PROGRAM PEMAGANGAN", 
                    "KURIKULUM PROGRAM PEMAGANGAN", "SILABUS PROGRAM PEMAGANGAN", "LAMPIRAN II",
                    "LAMPIRAN III", "   1. JADWAL PEMAGANGAN", "2. KEGIATAN HARIAN PESERTA YANG DIKETAHUI PEMBIMBING"
                    ]
    pages = [page for page in pages if not any(exclude_str in page for exclude_str in exclude_list)]
    text = '\x0c'.join(pages)
    
    # Split the text into paragraphs based on subheadings
    subheading_pattern = re.compile(r'^[A-Z]\.\s', re.MULTILINE)
    paragraphs = []
    
    matches = subheading_pattern.finditer(text)
    start = 0
    
    for match in matches:
        paragraph = text[start:match.start()].strip()
        paragraph = re.sub(r'\s+', ' ', paragraph)
        # Remove numbers between dashes
        paragraph = re.sub(r'-\s*\d+\s*-', ' ', paragraph)
        # Remove numbers followed by "http://jdih.pu.go.id"
        paragraph = re.sub(r'\d+\s*http://jdih\.pu\.go\.id', '', paragraph)
        # Remove Roman numerals
        paragraph = re.sub(r'\b[ivx]+\b', '', paragraph)
        paragraphs.append(paragraph)
        start = match.start()

    paragraph = text[start:].strip()
    paragraph = re.sub(r'\s+', ' ', paragraph)
    # Remove numbers between dashes
    paragraph = re.sub(r'-\s*\d+\s*-', ' ', paragraph)
    # Remove numbers followed by "http://jdih.pu.go.id"
    paragraph = re.sub(r'\d+\s*http://jdih\.pu\.go\.id', '', paragraph)
    # Remove Roman numerals
    paragraph = re.sub(r'\b[ivx]+\b', '', paragraph)
    paragraphs.append(paragraph)

    # Remove specific words from each paragraph
    remove_words = ["Dokumen ini tidak dikendalikan jika di unduh/Uncontrolled when downloaded", 
                    "Ditetapkan di Jakarta pada tanggal 6 Desember 2019 MENTERI PEKERJAAN UMUM DAN PERUMAHAN RAKYAT, ttd",
                    "M. BASUKI HADIMULJONO",
                    "www.djpp.depkumham.go.id",
                    "Format 2:",
                    "2. Tahap Pemagangan dan Evaluasi Pelaksanaan Pemagangan 8 9 10 11 SELESAI Persiapan Pemagangan Pelaksanaan Pemagangan Pengawasan Pelaksanaan Pemagangan Evaluasi Pelaksanaan Pemagangan",
                    "SALINAN"]
    for i in range(len(paragraphs)):
        for word in remove_words:
            paragraphs[i] = paragraphs[i].replace(word, "")
        
    return paragraphs

In [52]:
pdf_path = r"C:\Users\Annisa Rizki\Desktop\Annisa Lianda\Job Freelance\chatbot_using_openai\pdf\PermenPU13-2011.pdf"
paragraphs = extract_info(pdf_path)
paragraphs

['PERATURAN MENTERI PEKERJAAN UMUM REPUBLIK INDONESIA NOMOR 13/PRT/M/2011 TENTANG TATA CARA PEMELIHARAAN DAN PENILIKAN JALAN DENGAN RAHMAT TUHAN YANG MAHA ESA MENTERI PEKERJAAN UMUM REPUBLIK INDONESIA, Menimbang : Mengingat : bahwa untuk melaksanakan ketentuan Pasal 97, Pasal 101, pasal 104 dan Pasal 106 Peraturan Pemerintah Nomor 34 Tahun 2006 Tentang Jalan, perlu menetapkan Peraturan Menteri Pekerjaan Umum tentang Tata Cara Pemeliharaan Jalan dan Penilikan Jalan; 1. Undang Undang Republik Indonesia Nomor 22 tahun 2009 tentang Lalu Lintas dan Angkutan Jalan (Lembaran Negara Republik Indonesia tahun 2009 Nomor 96 dan Tambahan Lembaran Negara Republik Indonesia Nomor 2028); 2. Peraturan Pemerintah Nomor 34 Tahun 2006 tentang Jalan (Lembaran Negara Republik Indonesia Tahun 2006 Nomor 86, Tambahan Lembaran Negara Republik Indonesia Nomor 4655); 3. Peraturan Pemerintah Nomor 38 Tahun 2007 tentang Pembagian Urusan Pemerintah Antara Pemerintah, Pemerintah Daerah Provinsi,dan Pemerintah Daera